In [1]:
# from google.colab import drive
# drive.mount('/content/drive')

In [1]:
# !pip install python-resize-image

In [2]:
import numpy as np 
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from keras.preprocessing.image import img_to_array
from keras.preprocessing.image import array_to_img
from PIL import Image
import glob
import csv
import tensorflow as tf
from keras import regularizers
from resizeimage import resizeimage

Using TensorFlow backend.


In [3]:
def load_images(path):
    image_list = []
 
    count=0
    for filename in glob.glob(path): 
        im=Image.open(filename)
        #cover = resizeimage.resize_cover(im, [150, 200])
        #image_list.append([filename[::-1][:filename[::-1].find('\\')][::-1],cover])
        image_list.append([filename[::-1][:filename[::-1].find('\\')][::-1],im])
        count+=1
        
    #print(count)
    #print(len(image_list))
    #print(image_list[5])
    
    return image_list

In [30]:
def array_to_image(img_list):
    img_array = []
    for i in range(len(img_list)):
        cover = array_to_img(img_list[i])
        cover = resizeimage.resize_cover(cover, [100, 150])
        img_array.append(img_to_array(cover))
       
  
    img_array = np.array(img_array)
    
    return img_array

In [4]:
def image_to_array(img_list):
    img_array = []
    for i in range(len(img_list)):
        img_array.append(img_to_array(img_list[i][1]))

    img_array = np.array(img_array)
    
    return img_array

In [5]:
def parse_meta_data(path):
    rows = []
    with open(path) as csv_file:
        csv_reader = csv.reader(csv_file, delimiter=',')
        line_count = 0
        for row in csv_reader:
            if line_count == 0:
                  line_count += 1
            else:
                  rows.append(row)

    meta_data = {}
    for i in range(len(rows)):
        meta_data[rows[i][0]+'.bmp'] = rows[i][-1]
      
    return meta_data

In [6]:
def get_train_data(meta_data,img_list):
    ratings = []
    img_array = []
    for i in range(len(img_list)):
        ratings.append(float(meta_data[img_list[i][0]]))
        img_array.append(img_to_array(img_list[i][1]))
    ratings = np.array(ratings)
    img_array = np.array(img_array)
    return img_array,ratings

In [7]:
train_image_list = load_images('StudentData/train/*.bmp')
test_image_list = load_images('StudentData/test/*.bmp')
train_meta_data = parse_meta_data('StudentData/train_scores.csv')

x_train,y_train = get_train_data(train_meta_data,train_image_list)
x_test = image_to_array(test_image_list)

In [8]:
x_train.shape

(197, 710, 512, 1)

In [9]:
train_image_list[0] 

['0024ad48c2d69a9da845b4052a000803e7391163.bmp',
 <PIL.BmpImagePlugin.BmpImageFile image mode=L size=512x710 at 0x1CB7FC082E8>]

In [10]:
# x_test[0].shape
x_train.shape

(197, 710, 512, 1)

In [11]:
x_train[:][:][196]

array([[[ 19.],
        [ 20.],
        [ 23.],
        ...,
        [  0.],
        [  0.],
        [  0.]],

       [[ 21.],
        [ 20.],
        [ 21.],
        ...,
        [  0.],
        [  0.],
        [  0.]],

       [[ 18.],
        [ 19.],
        [ 18.],
        ...,
        [  0.],
        [  0.],
        [  0.]],

       ...,

       [[ 30.],
        [ 34.],
        [ 38.],
        ...,
        [  0.],
        [  0.],
        [  0.]],

       [[ 64.],
        [ 80.],
        [ 99.],
        ...,
        [  0.],
        [  0.],
        [  0.]],

       [[255.],
        [255.],
        [255.],
        ...,
        [  0.],
        [  0.],
        [  0.]]], dtype=float32)

In [20]:
def cropImage(im):
    # brute force state function to find size of left pipe


    allWidth = list()
    allStart = list()

    allPipeWidth = []
    
    for row in range(0,710):

        done = False
        col = 511

        while not done:

            if im[row, col] != 0:

                allStart.append(col)
                inPipe = True
                n = 1

                while inPipe:

                    col -= 1

                    if 0.7*im[row,col+1] <= im[row,col] <= 1.3*im[row,col+1]:
                        n += 1
                    else:
                        inPipe = False
                        allPipeWidth.append(n) 
                        done = True

            else:
                col -= 1        
                
    mean = np.average(allPipeWidth)
    std = np.std(allPipeWidth)
    #print(mean, std)

    # I know the answer is around 45 from image processing, so let's remove some outliers

    M = len(allPipeWidth)
    n = 0
    while n < M:
        if allPipeWidth[n] <= (mean - std) or allPipeWidth[n] >= (mean + std):
            del allPipeWidth[n]
        M = len(allPipeWidth)
        n +=1

    mean = np.average(allPipeWidth)
    std = np.std(allPipeWidth)
    #print(mean, std)

    for row in range(0,710):
        im[row,allStart[row]-int(mean+1):allStart[row]] = 0.0

    return im

In [39]:
def CropImageandResize(x_train):
    x_train_cropped = []
    for i in range(0, x_train.shape[0]):
        ele = np.squeeze(x_train[:][:][i], axis=2)
        ele_cropped = cropImage(ele)
        ele_3d = np.expand_dims(ele_cropped, axis=2)
        x_train_cropped.append(ele_3d)
    
    x_train_cropped = np.asarray(x_train_cropped)
    x_train_cr = array_to_image(x_train_cropped)
    
    return x_train_cr
    

In [40]:
x_train_t = CropImageandResize(x_train)

In [41]:
x_test_t = CropImageandResize(x_test)

In [42]:
x_train_t.shape

(197, 150, 100, 1)

In [43]:
x_test_t.shape

(99, 150, 100, 1)

In [22]:
print(len(x_train_cropped), len(x_train_cropped[0]), len(x_train_cropped[0][0]), len(x_train_cropped[0][0][0]))

197 710 512 1


In [24]:
x_train_cropped.shape

(197, 710, 512, 1)

In [32]:
x_train_cr.shape

(197, 150, 100, 1)

In [36]:
model1=tf.keras.models.Sequential([
    
    tf.keras.layers.Conv2D(64,(3,3),activation='relu',input_shape=(150,100,1)),
    tf.keras.layers.Conv2D(64,(3,3),activation='relu'),
    tf.keras.layers.MaxPooling2D(pool_size=(3,3),strides=[3,3]),

    tf.keras.layers.Conv2D(32,(3,3),activation='relu'),
    tf.keras.layers.Conv2D(32,(3,3),activation='relu'),
    tf.keras.layers.MaxPooling2D(pool_size=(2,2),strides=[2,2]),
    #tf.keras.layers.Dropout(0.4),

    tf.keras.layers.Flatten(),
       
    tf.keras.layers.Dense(256,activation='relu'),
    tf.keras.layers.Dense(128,activation='relu'),
    #tf.keras.layers.Dropout(0.4),
    tf.keras.layers.Dense(1)
])

In [37]:
reduce =tf. keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=10, mode='auto')
loss=tf.keras.losses.Huber()
#early = tf.keras.callbacks.EarlyStopping(monitor='val_loss', min_delta=1e-4, patience=10, mode='auto')
model1.compile(optimizer='adam',loss='mean_squared_error',metrics=['mae'])
model1.fit(x_train_cr, y_train, batch_size=32, epochs=50, verbose=1, callbacks=None, validation_split=0.05, shuffle=True,use_multiprocessing=True)

Train on 187 samples, validate on 10 samples
Epoch 1/50
187/187 [==============================] - 12s 65ms/sample - loss: 46.7418 - mean_absolute_error: 5.0138 - val_loss: 11.0477 - val_mean_absolute_error: 2.6439
Epoch 2/50
187/187 [==============================] - 11s 60ms/sample - loss: 5.9786 - mean_absolute_error: 1.9944 - val_loss: 3.5272 - val_mean_absolute_error: 1.5769
Epoch 3/50
187/187 [==============================] - 12s 65ms/sample - loss: 4.5269 - mean_absolute_error: 1.7059 - val_loss: 4.1760 - val_mean_absolute_error: 1.6858
Epoch 4/50
187/187 [==============================] - 11s 58ms/sample - loss: 3.6224 - mean_absolute_error: 1.5553 - val_loss: 2.6099 - val_mean_absolute_error: 1.4270
Epoch 5/50
187/187 [==============================] - 11s 60ms/sample - loss: 3.0880 - mean_absolute_error: 1.4280 - val_loss: 2.6867 - val_mean_absolute_error: 1.2851
Epoch 6/50
187/187 [==============================] - 11s 60ms/sample - loss: 2.6822 - mean_absolute_error: 1.315

In [45]:
train_pred = model1.predict(x_train_cr)

In [46]:
true_mean = np.mean(y_train)
upper = 0
lower = 0
for i in range(len(y_train)):
    upper = upper + (np.square(y_train[i]-train_pred[i]))
    lower = lower + (np.square(y_train[i] - true_mean))
r_squared = 1 - (upper[0]/lower)

In [47]:
r_squared

0.9896748038936335

In [50]:
test_pred = model1.predict(x_test_t)

In [51]:
with open('team13submission.csv', 'w') as file:
    file.write('image,total_rating\n')
    for i in range(len(x_test)):
        file.write(str(test_image_list[i][0][:-4])+','+str(test_pred[i][0])+'\n')